In [1]:
# Required libraries
import sys
import datetime
import time 

from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import *
from pyspark.sql import functions as sFuncs
from pyspark.sql.window import Window

### Note: 
In case you are using a single-node cluster, executing this cell is essential, as otherwise, SparkContext put the sc.master on 'yarn' which you don't have. The result would be you'll never see a collect() to converge.

In [ ]:
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("QA4") \
    .getOrCreate()

## HW3 Part2

In [ ]:
df = spark.read.format("csv").option('header','true').load('gs://datathinks-home/online_retail_II.csv')
df.show(10)

### Data preprocessing

In [ ]:
# select\rename columns
df = df.withColumn("CustomerID", df["Customer ID"]).drop("Customer ID", "Country", "Description")
n_records_before_cleaning = df.count()
# remove duplicate records
df = df.distinct()
n_records_after_removing_duplicates = df.count()
print('Number of records befor duplicate removal: ', n_records_before_cleaning)
print('Number of records after duplicate removal: ', n_records_after_removing_duplicates)
print('Number of duplicate records removed: ', n_records_before_cleaning-n_records_after_removing_duplicates)
# Missing Value Imputation
n_null_records = df.filter('CustomerID is null').count()
print('Number of Null records: ', n_null_records)
print('Number of records befor missing value imputation: ', df.count())
df = df.na.drop()
print('Number of records after missing value imputation: ', df.count())

df.show(5)

### 2. Calcuate Monetary Value

In [ ]:
df = df.withColumn("Price", df.Price.cast('float'))
df.printSchema()

In [ ]:
monDF = df.groupBy('customerID').agg(sum('Price').alias('Monetary'))
monDF = monDF.orderBy(desc('Monetary'))
tmp= monDF.withColumn("new_column",lit("ABC"))
w = Window().partitionBy('new_column').orderBy(lit('A'))
monDF = tmp.withColumn("id", row_number().over(w)).drop("new_column")

monDF.show(10)

In [ ]:
def func(pair):
    (key, val) = pair
    result = 0
    if val < 0.15*cnt: 
        result = 1
    if 0.15*cnt < val < 0.3*cnt: 
        result = 2
    if 0.3*cnt < val < 0.6*cnt: 
        result = 3
    if 0.6*cnt < val : 
        result = 4
    return (key, result)

cnt = monDF.count()
monRDD = monDF.rdd
monRDD = monRDD.map(lambda x:(x[0],x[2])).map(func)
# monRDD.collect()

In [ ]:
# RDD code to compute aggregate average
monDF = monRDD.toDF(["customerID", "Monetary"])
monDF.show(10)

### 3. Calcuate Frequency

In [ ]:
frqDF = df.groupBy('customerID').agg(count('invoice').alias('Frequency'))
frqDF = frqDF.orderBy(desc('Frequency'))
tmp= frqDF.withColumn("new_column",lit("ABC"))
w = Window().partitionBy('new_column').orderBy(lit('A'))
frqDF = tmp.withColumn("id", row_number().over(w)).drop("new_column")

frqDF.show(10)

In [ ]:
cnt = frqDF.count()
frqRDD = frqDF.rdd
frqRDD = frqRDD.map(lambda x:(x[0],x[2])).map(func)
# frqRDD.collect()

In [ ]:
# RDD code to compute aggregate average
frqDF = frqRDD.toDF(["customerID", "Frequency"])
frqDF.show(10)

### 4. Calcuate Recency

In [ ]:
from datetime import datetime as dt

In [ ]:
# Setting an user define function:
# This function converts the string cell into a date:
dateFormat =  udf (lambda x: dt.strptime(x, '%m/%d/%Y %H:%M'), DateType())

recDF = df.groupBy('CustomerID').agg(max('InvoiceDate').alias('Recency'))
recDF = recDF.select('CustomerID','Recency') \
        .withColumn('Recency', dateFormat(col('Recency')))
recDF = recDF.orderBy(desc('Recency'))
recDF.show(10)

In [ ]:
def compDate(pair):
    (key, val) = pair
    result = 0
    if dt.date(dt(2011, 11, 15)) <= val: 
        result = 1
    if dt.date(dt(2011, 9, 5)) <= val <= dt.date(dt(2011, 11, 14)): 
        result = 2
    if dt.date(dt(2011, 1, 5)) <= val <= dt.date(dt(2011, 9, 4)): 
        result = 3
    if val <= dt.date(dt(2011, 1, 4)): 
        result = 4
    return (key, result)


# func =  udf (compDate, DateType())
# recDF.withColumn('Recency', func(col('Recency'))).show(10)

recRDD = recDF.rdd
recRDD = recRDD.map(lambda x:(x[0],x[1])).map(compDate)
# recRDD.collect()

In [ ]:
# RDD code to compute aggregate average
recDF = recRDD.toDF(["customerID", "Recency"])
recDF.show(10)

### 5. Number of customers in each category


In [ ]:
uDF = recDF.join(frqDF, on=['CustomerID'], how='outer')
uDF = uDF.join(monDF, on=['CustomerID'], how='outer')
uDF.show(10)

In [ ]:
BestCustomer = uDF.where((col('Recency')==lit('1')) \
            & (col('Frequency')==lit('1')) \
            & (col('Monetary')==lit('1'))).count()

LoyalCustomer = uDF.where((col('Frequency')==lit('1'))).count()

BigSpender = uDF.where((col('Monetary')==lit('1'))).count()

AlmostLost = uDF.where((col('Recency')==lit('3')) \
            & (col('Frequency')==lit('1')) \
            & (col('Monetary')==lit('1'))).count()

LostCustomers = uDF.where((col('Recency')==lit('4')) \
            & (col('Frequency')==lit('1')) \
            & (col('Monetary')==lit('1'))).count()

LostCheapCustomers = uDF.where((col('Recency')==lit('4')) \
            & (col('Frequency')==lit('4')) \
            & (col('Monetary')==lit('4'))).count()

In [ ]:
print("Number of Best Customers: ", BestCustomer)
print("Number of Loyal Customers: ", LoyalCustomer)
print("Number of Big Spenders: ", BigSpender)
print("Number of Almost Lost: ", AlmostLost)
print("Number of Lost Customers: ", LostCustomers)
print("Number of Lost Cheap Customers: ", LostCheapCustomers)

## PA4 Question 4

A freshly-loaded copy of the NYT covid dataset is available as [gs://datathinks-home/covid2.json](gs://datathinks-home/covid2.json).

Please don’t upload your data! Instead, starting on March 1, 2020, for the first day of each month, which county had the worst numbers of confirmed cases, and deaths? They might not be the same county. In other words, develop a table that looks like this:

| Query | 4/1 | 5/1 | ...etc... |
| --- | --- | --- | --- |
| Confirmed Cases | nnn, County, State | nnn, County, State |   |
| Deaths | nnn, County, State | nnn, County, State |   |

This analysis should be done on Spark.

## PA4 Question 1

The last 5 presidents in our speeches collection were `reagan, bush, clinton, gwbush` and `obama` (that's how the tar.gz files of their speeches are named in Canvas).

Pairwise comparisons of the similarities in their speech collections (10 pairs) will give us a half-matrix like shown below (The symmetry in the problem formulation makes it unnecessary to compute the blank spots in the matrix):

|  | r | b | c | g | o |
| --- | --- | --- | --- | --- | --- |
| reagan |   | * | * | * | * |
| bush |   |   | * | * | * |
| clinton |   |   |   | * | * |
| gwbush |   |   |   |   | * |

Compute the similarity denoted by each asterisk and answer:

1. Using n-gram character shingles, assuming n=4, which two presidents' speeches were the most similar and which were the least similar?
2. Using n-gram word shingles, assuming n=3, which two presidents' speeches were the most similar and which were the least similar?

## PA4 Question 3

This question builds on the [UCI Online Retail II dataset](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II) analysis you performed in Quiz 3. This time, however, the R, F, M values should be calculated as follows:

Recency should be the number of days relative to year-end 2011 (Dec 31). 
Frequency should simply be the number of transactions in the total period.
Monetary value should be the log10 of the total dollars spent. Why log10? We use logs to flatten the range — so high-spenders don't skew the analysis.
After calculating RFM values as specified above, run K-means clustering to divide the customers into 6 clusters. How do the number of customers in these 6 clusters compare with the clusters you in Question 2 of Quiz 3?

## PA4 Question 4

A freshly-loaded copy of the NYT covid dataset is available as [`gs://datathinks-home/covid2.json`](gs://datathinks-home/covid2.json).

Please don’t upload your data! Instead, starting on March 1, 2020, for the first day of each month, which county had the worst numbers of confirmed cases, and deaths? They might not be the same county. In other words, develop a table that looks like this:

| Query | 4/1 | 5/1 | ...etc... |
| --- | --- | --- | --- |
| Confirmed Cases | nnn, County, State | nnn, County, State |   |
| Deaths | nnn, County, State | nnn, County, State |   |

This analysis should be done on Spark.

In [ ]:
# to read the .json file
covidDF = spark.read.json('gs://datathinks-home/covid2.json')
# covidDF = spark.read.load('gs://datathinks-home/covid2.json', 
#                           format='json', inferSchema='true')#, header='true')

In [ ]:
covidDF.show(10)

In [ ]:
covidDF.printSchema()

In [ ]:
# there is no missing value in columns other than county_fips_code column which we can drop
for col in covidDF.columns:
    condition = '{} is null'.format(col)
    print('number of nulls in {:s}: {:>10}'.format(col, covidDF.filter(condition).count()))

In [ ]:
# to 
covidDF = covidDF.withColumn('confirmed_cases', covidDF.confirmed_cases.cast('int'))\
                 .withColumn('deaths', covidDF.deaths.cast('int'))\
                 .withColumn('date', to_date(covidDF.date))

dateDF = covidDF.filter(sFuncs.dayofmonth(covidDF.date) == 1).sort('date')

In [ ]:
def findStat(num, county, state):
    return str(num) + ', ' + str(county) + ', ' + str(state)

# def findStat(*row):
#     return ','.join(row)

func = sFuncs.udf(findStat)

start = time.time()
max_cases = dateDF.sort(desc('confirmed_cases'))\
                  .groupBy('date')\
                  .agg(sFuncs.max('confirmed_cases').alias('cases'), \
                       first('county').alias('county'), \
                       first('state_name').alias('state'))\
                  .select(sFuncs.date_format('date', 'MM/dd').alias('date'), \
                          func('cases', 'county', 'state').alias('max_cases'))

max_deaths = dateDF.sort(desc('deaths'))\
                   .groupBy('date')\
                   .agg(sFuncs.max('deaths').alias('deaths'), \
                        first('county').alias('county'), \
                        first('state_name').alias('state'))\
                   .select(sFuncs.date_format('date', 'MM/dd').alias('date'), \
                           func('deaths', 'county', 'state').alias('max_deaths'))

max_cases.join(max_deaths, ['date']).sort('date').show(truncate=False)

print('elapsed time: {:.0f} seconds'.format(time.time() - start))

In [ ]:
byDate = Window.partitionBy('date').orderBy(desc('deaths'))
dateDF.withColumn('rank', dense_rank().over(byDate)).filter('rank == 1').show()